In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from src.env.dunderbot_env import DunderBotEnv

# TODO:
- remove the bias when getting the observations (suggestion in the repo issues)
- exchange data to something more interesting, consider implementing time steps
- modularize actions, rewards, and observations (look at https://github.com/notadamking/RLTrader for (kinda) readable examples)
- understand the rendering, especially during learning
- understand the time steps in learn and predict


# DONE: 
- add config

# Run main

In [10]:
import gym
import json
import datetime as dt

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2

In [11]:
from src.env.dunderbot_env import DunderBotEnv

import pandas as pd

df = pd.read_csv('./data/AAPL.csv')
df = df.sort_values('Date')
df = df.drop(columns=['Unnamed: 0'])

In [12]:
# The algorithms require a vectorized environment to run
env = DummyVecEnv([lambda: DunderBotEnv(df)])

model = PPO2(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=20000)

----------------------------------------
| approxkl           | 2.515224e-07    |
| clipfrac           | 0.0             |
| explained_variance | -2.38e-06       |
| fps                | 173             |
| n_updates          | 1               |
| policy_entropy     | 2.8379526       |
| policy_loss        | -0.000100153615 |
| serial_timesteps   | 128             |
| time_elapsed       | 3.03e-05        |
| total_timesteps    | 128             |
| value_loss         | 4988355.5       |
----------------------------------------


/code/src/env/dunderbot_env.py:77: RuntimeWarning: invalid value encountered in double_scalars
  prev_cost + additional_cost) / (self.shares_held + shares_bought)


--------------------------------------
| approxkl           | 1.7056612e-07 |
| clipfrac           | 0.0           |
| explained_variance | -5.96e-07     |
| fps                | 347           |
| n_updates          | 2             |
| policy_entropy     | 2.8383095     |
| policy_loss        | -4.879583e-05 |
| serial_timesteps   | 256           |
| time_elapsed       | 0.74          |
| total_timesteps    | 256           |
| value_loss         | 13683891.0    |
--------------------------------------
--------------------------------------
| approxkl           | 1.8449879e-07 |
| clipfrac           | 0.0           |
| explained_variance | -8.34e-07     |
| fps                | 307           |
| n_updates          | 3             |
| policy_entropy     | 2.839201      |
| policy_loss        | -7.352908e-05 |
| serial_timesteps   | 384           |
| time_elapsed       | 1.11          |
| total_timesteps    | 384           |
| value_loss         | 17578868.0    |
-------------------------

---------------------------------------
| approxkl           | 4.4779695e-06  |
| clipfrac           | 0.0            |
| explained_variance | -2.72e-05      |
| fps                | 331            |
| n_updates          | 19             |
| policy_entropy     | 2.8516016      |
| policy_loss        | -0.00019003963 |
| serial_timesteps   | 2432           |
| time_elapsed       | 7.31           |
| total_timesteps    | 2432           |
| value_loss         | 65243.094      |
---------------------------------------
--------------------------------------
| approxkl           | 9.5687574e-05 |
| clipfrac           | 0.0           |
| explained_variance | -0.000549     |
| fps                | 345           |
| n_updates          | 20            |
| policy_entropy     | 2.8521376     |
| policy_loss        | -0.0008009323 |
| serial_timesteps   | 2560          |
| time_elapsed       | 7.7           |
| total_timesteps    | 2560          |
| value_loss         | 106138.08     |
------------

---------------------------------------
| approxkl           | 0.00016329288  |
| clipfrac           | 0.0            |
| explained_variance | 1.35e-05       |
| fps                | 336            |
| n_updates          | 36             |
| policy_entropy     | 2.8587217      |
| policy_loss        | -0.00034248806 |
| serial_timesteps   | 4608           |
| time_elapsed       | 13.9           |
| total_timesteps    | 4608           |
| value_loss         | 102309464.0    |
---------------------------------------
--------------------------------------
| approxkl           | 0.00024451566 |
| clipfrac           | 0.0           |
| explained_variance | 1.4e-05       |
| fps                | 349           |
| n_updates          | 37            |
| policy_entropy     | 2.8589337     |
| policy_loss        | 0.0016426364  |
| serial_timesteps   | 4736          |
| time_elapsed       | 14.3          |
| total_timesteps    | 4736          |
| value_loss         | 102662200.0   |
------------

...

-------------------------------------
| approxkl           | 0.0011817459 |
| clipfrac           | 0.001953125  |
| explained_variance | 8.34e-07     |
| fps                | 334          |
| n_updates          | 87           |
| policy_entropy     | 2.8639364    |
| policy_loss        | 0.006527992  |
| serial_timesteps   | 11136        |
| time_elapsed       | 33.1         |
| total_timesteps    | 11136        |
| value_loss         | 214182830.0  |
-------------------------------------
--------------------------------------
| approxkl           | 0.0002434966  |
| clipfrac           | 0.0           |
| explained_variance | -2.62e-06     |
| fps                | 365           |
| n_updates          | 88            |
| policy_entropy     | 2.862677      |
| policy_loss        | -0.0007722962 |
| serial_timesteps   | 11264         |
| time_elapsed       | 33.5          |
| total_timesteps    | 11264         |
| value_loss         | 168501580.0   |
--------------------------------------

-------------------------------------
| approxkl           | 0.0025729183 |
| clipfrac           | 0.015625     |
| explained_variance | -5.48e-06    |
| fps                | 344          |
| n_updates          | 104          |
| policy_entropy     | 2.8688297    |
| policy_loss        | 0.0011416719 |
| serial_timesteps   | 13312        |
| time_elapsed       | 39.4         |
| total_timesteps    | 13312        |
| value_loss         | 381813280.0  |
-------------------------------------
--------------------------------------
| approxkl           | 0.00017852259 |
| clipfrac           | 0.0           |
| explained_variance | 9.54e-07      |
| fps                | 355           |
| n_updates          | 105           |
| policy_entropy     | 2.8685098     |
| policy_loss        | -0.0005806945 |
| serial_timesteps   | 13440         |
| time_elapsed       | 39.8          |
| total_timesteps    | 13440         |
| value_loss         | 554927500.0   |
--------------------------------------

-------------------------------------
| approxkl           | 0.0001467797 |
| clipfrac           | 0.0          |
| explained_variance | 2.15e-06     |
| fps                | 372          |
| n_updates          | 121          |
| policy_entropy     | 2.8685167    |
| policy_loss        | 0.0005551097 |
| serial_timesteps   | 15488        |
| time_elapsed       | 45.5         |
| total_timesteps    | 15488        |
| value_loss         | 1816147000.0 |
-------------------------------------
--------------------------------------
| approxkl           | 0.00038830884 |
| clipfrac           | 0.0           |
| explained_variance | 2.98e-07      |
| fps                | 368           |
| n_updates          | 122           |
| policy_entropy     | 2.8688383     |
| policy_loss        | -0.002679212  |
| serial_timesteps   | 15616         |
| time_elapsed       | 45.9          |
| total_timesteps    | 15616         |
| value_loss         | 1903695600.0  |
--------------------------------------

--------------------------------------
| approxkl           | 4.400767e-05  |
| clipfrac           | 0.0           |
| explained_variance | 1.79e-06      |
| fps                | 344           |
| n_updates          | 138           |
| policy_entropy     | 2.8682766     |
| policy_loss        | -0.0003068305 |
| serial_timesteps   | 17664         |
| time_elapsed       | 51.6          |
| total_timesteps    | 17664         |
| value_loss         | 47021852.0    |
--------------------------------------
--------------------------------------
| approxkl           | 0.0013238887  |
| clipfrac           | 0.00390625    |
| explained_variance | 2.98e-07      |
| fps                | 353           |
| n_updates          | 139           |
| policy_entropy     | 2.867813      |
| policy_loss        | -0.0015173261 |
| serial_timesteps   | 17792         |
| time_elapsed       | 52            |
| total_timesteps    | 17792         |
| value_loss         | 54756580.0    |
-------------------------

--------------------------------------
| approxkl           | 0.00048045852 |
| clipfrac           | 0.0           |
| explained_variance | 0             |
| fps                | 368           |
| n_updates          | 155           |
| policy_entropy     | 2.8693876     |
| policy_loss        | 0.0013406577  |
| serial_timesteps   | 19840         |
| time_elapsed       | 57.8          |
| total_timesteps    | 19840         |
| value_loss         | 6291649500.0  |
--------------------------------------
---------------------------------------
| approxkl           | 0.00019440048  |
| clipfrac           | 0.0            |
| explained_variance | -1.19e-07      |
| fps                | 370            |
| n_updates          | 156            |
| policy_entropy     | 2.8701253      |
| policy_loss        | -0.00056981714 |
| serial_timesteps   | 19968          |
| time_elapsed       | 58.1           |
| total_timesteps    | 19968          |
| value_loss         | 6804702000.0   |
-------------

In [13]:
obs = env.reset()
for i in range(2000):
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()

Step: 68
Balance: 10000
Shares held: 0 (Total sold: 0)
Avg cost for held shares: 0 (Total sales value: 0)
Net worth: 10000.0 (Max net worth: 10000)
Profit: 0.0
Step: 69
Balance: 10000
Shares held: 0 (Total sold: 0)
Avg cost for held shares: 0 (Total sales value: 0)
Net worth: 10000.0 (Max net worth: 10000)
Profit: 0.0
Step: 70
Balance: 10000
Shares held: 0 (Total sold: 0)
Avg cost for held shares: 0 (Total sales value: 0)
Net worth: 10000.0 (Max net worth: 10000)
Profit: 0.0
Step: 71
Balance: 10000
Shares held: 0 (Total sold: 0)
Avg cost for held shares: 0 (Total sales value: 0)
Net worth: 10000.0 (Max net worth: 10000)
Profit: 0.0
Step: 72
Balance: 10000
Shares held: 0 (Total sold: 0)
Avg cost for held shares: 0 (Total sales value: 0)
Net worth: 10000.0 (Max net worth: 10000)
Profit: 0.0
Step: 73
Balance: 10000
Shares held: 0 (Total sold: 0)
Avg cost for held shares: 0 (Total sales value: 0)
Net worth: 10000.0 (Max net worth: 10000)
Profit: 0.0
Step: 74
Balance: 10000.0
Shares held: 0

Step: 119
Balance: 10000.0
Shares held: 0 (Total sold: 0)
Avg cost for held shares: 0 (Total sales value: 0.0)
Net worth: 10000.0 (Max net worth: 10000)
Profit: 0.0
Step: 120
Balance: 10000.0
Shares held: 0 (Total sold: 0)
Avg cost for held shares: 0 (Total sales value: 0.0)
Net worth: 10000.0 (Max net worth: 10000)
Profit: 0.0
Step: 121
Balance: 10000.0
Shares held: 0 (Total sold: 0)
Avg cost for held shares: 0 (Total sales value: 0.0)
Net worth: 10000.0 (Max net worth: 10000)
Profit: 0.0
Step: 122
Balance: 10000.0
Shares held: 0 (Total sold: 0)
Avg cost for held shares: 0 (Total sales value: 0.0)
Net worth: 10000.0 (Max net worth: 10000)
Profit: 0.0
Step: 123
Balance: 10000.0
Shares held: 0 (Total sold: 0)
Avg cost for held shares: 0 (Total sales value: 0.0)
Net worth: 10000.0 (Max net worth: 10000)
Profit: 0.0
Step: 124
Balance: 10000.0
Shares held: 0 (Total sold: 0)
Avg cost for held shares: 0 (Total sales value: 0.0)
Net worth: 10000.0 (Max net worth: 10000)
Profit: 0.0
Step: 125


Step: 171
Balance: 10372.349137207064
Shares held: 0 (Total sold: 59)
Avg cost for held shares: 0 (Total sales value: 2470.0216278984662)
Net worth: 10372.349137207064 (Max net worth: 10372.349137207064)
Profit: 372.3491372070639
Step: 172
Balance: 10372.349137207064
Shares held: 0 (Total sold: 59)
Avg cost for held shares: 0 (Total sales value: 2470.0216278984662)
Net worth: 10372.349137207064 (Max net worth: 10372.349137207064)
Profit: 372.3491372070639
Step: 173
Balance: 10372.349137207064
Shares held: 0 (Total sold: 59)
Avg cost for held shares: 0 (Total sales value: 2470.0216278984662)
Net worth: 10372.349137207064 (Max net worth: 10372.349137207064)
Profit: 372.3491372070639
Step: 174
Balance: 10372.349137207064
Shares held: 0 (Total sold: 59)
Avg cost for held shares: 0 (Total sales value: 2470.0216278984662)
Net worth: 10372.349137207064 (Max net worth: 10372.349137207064)
Profit: 372.3491372070639
Step: 175
Balance: 10372.349137207064
Shares held: 0 (Total sold: 59)
Avg cost f

Step: 225
Balance: 10372.349137207064
Shares held: 0 (Total sold: 59)
Avg cost for held shares: 0 (Total sales value: 2470.0216278984662)
Net worth: 10372.349137207064 (Max net worth: 10372.349137207064)
Profit: 372.3491372070639
Step: 226
Balance: 10372.349137207064
Shares held: 0 (Total sold: 59)
Avg cost for held shares: 0 (Total sales value: 2470.0216278984662)
Net worth: 10372.349137207064 (Max net worth: 10372.349137207064)
Profit: 372.3491372070639
Step: 227
Balance: 10372.349137207064
Shares held: 0 (Total sold: 59)
Avg cost for held shares: 0 (Total sales value: 2470.0216278984662)
Net worth: 10372.349137207064 (Max net worth: 10372.349137207064)
Profit: 372.3491372070639
Step: 228
Balance: 10372.349137207064
Shares held: 0 (Total sold: 59)
Avg cost for held shares: 0 (Total sales value: 2470.0216278984662)
Net worth: 10372.349137207064 (Max net worth: 10372.349137207064)
Profit: 372.3491372070639
Step: 229
Balance: 10372.349137207064
Shares held: 0 (Total sold: 59)
Avg cost f

Step: 277
Balance: 9219.029137207064
Shares held: 28 (Total sold: 59)
Avg cost for held shares: 41.19 (Total sales value: 2470.0216278984662)
Net worth: 10257.259415994216 (Max net worth: 10379.09732192737)
Profit: 257.2594159942164
Step: 278
Balance: 9219.029137207064
Shares held: 28 (Total sold: 59)
Avg cost for held shares: 41.19 (Total sales value: 2470.0216278984662)
Net worth: 10260.828807261814 (Max net worth: 10379.09732192737)
Profit: 260.8288072618143
Step: 279
Balance: 9219.029137207064
Shares held: 28 (Total sold: 59)
Avg cost for held shares: 41.19 (Total sales value: 2470.0216278984662)
Net worth: 10286.226862873888 (Max net worth: 10379.09732192737)
Profit: 286.2268628738875
Step: 280
Balance: 9219.029137207064
Shares held: 28 (Total sold: 59)
Avg cost for held shares: 41.19 (Total sales value: 2470.0216278984662)
Net worth: 10318.805229311138 (Max net worth: 10379.09732192737)
Profit: 318.80522931113774
Step: 281
Balance: 9219.029137207064
Shares held: 28 (Total sold: 5

...

Step: 1861
Balance: 1421.8999426884457
Shares held: 189 (Total sold: 5451)
Avg cost for held shares: 38.702188125935045 (Total sales value: 134342.4958973128)
Net worth: 9021.570426589544 (Max net worth: 17114.474096471095)
Profit: -978.4295734104562
Step: 1862
Balance: 1421.8999426884457
Shares held: 189 (Total sold: 5451)
Avg cost for held shares: 38.702188125935045 (Total sales value: 134342.4958973128)
Net worth: 9094.677356155073 (Max net worth: 17114.474096471095)
Profit: -905.3226438449274
Step: 1863
Balance: 4301.455457665308
Shares held: 117 (Total sold: 5523)
Avg cost for held shares: 38.702188125935045 (Total sales value: 137222.05141228967)
Net worth: 8980.73316950271 (Max net worth: 17114.474096471095)
Profit: -1019.2668304972904
Step: 1864
Balance: 4301.455457665308
Shares held: 117 (Total sold: 5523)
Avg cost for held shares: 38.702188125935045 (Total sales value: 137222.05141228967)
Net worth: 9006.54655989082 (Max net worth: 17114.474096471095)
Profit: -993.45344010918

Step: 1913
Balance: 7746.879070444535
Shares held: 52 (Total sold: 5775)
Avg cost for held shares: 38.32302095079176 (Total sales value: 147817.95492151091)
Net worth: 10025.178290872358 (Max net worth: 17114.474096471095)
Profit: 25.178290872358048
Step: 1914
Balance: 7746.879070444535
Shares held: 52 (Total sold: 5775)
Avg cost for held shares: 38.32302095079176 (Total sales value: 147817.95492151091)
Net worth: 10031.847023702265 (Max net worth: 17114.474096471095)
Profit: 31.847023702264778
Step: 1915
Balance: 7746.879070444535
Shares held: 52 (Total sold: 5775)
Avg cost for held shares: 38.32302095079176 (Total sales value: 147817.95492151091)
Net worth: 10093.425692347602 (Max net worth: 17114.474096471095)
Profit: 93.42569234760231
Step: 1916
Balance: 7746.879070444535
Shares held: 52 (Total sold: 5775)
Avg cost for held shares: 38.32302095079176 (Total sales value: 147817.95492151091)
Net worth: 10209.730037049063 (Max net worth: 17114.474096471095)
Profit: 209.73003704906296
S

Step: 1958
Balance: 698.2807459608994
Shares held: 183 (Total sold: 5801)
Avg cost for held shares: 50.55212892579608 (Total sales value: 149023.9976457274)
Net worth: 10084.508614098344 (Max net worth: 17114.474096471095)
Profit: 84.50861409834397
Step: 1959
Balance: 698.2807459608994
Shares held: 183 (Total sold: 5801)
Avg cost for held shares: 50.55212892579608 (Total sales value: 149023.9976457274)
Net worth: 10584.953888861397 (Max net worth: 17114.474096471095)
Profit: 584.953888861397
Step: 1960
Balance: 698.2807459608994
Shares held: 183 (Total sold: 5801)
Avg cost for held shares: 50.55212892579608 (Total sales value: 149023.9976457274)
Net worth: 10478.95083029286 (Max net worth: 17114.474096471095)
Profit: 478.9508302928607
Step: 1961
Balance: 698.2807459608994
Shares held: 183 (Total sold: 5801)
Avg cost for held shares: 50.55212892579608 (Total sales value: 149023.9976457274)
Net worth: 10345.041848185145 (Max net worth: 17114.474096471095)
Profit: 345.0418481851448
Step: 

Shares held: 46 (Total sold: 5938)
Avg cost for held shares: 50.55212892579608 (Total sales value: 157850.20113646373)
Net worth: 12859.504675829772 (Max net worth: 17114.474096471095)
Profit: 2859.504675829772
Step: 2003
Balance: 9524.484236697228
Shares held: 46 (Total sold: 5938)
Avg cost for held shares: 50.55212892579608 (Total sales value: 157850.20113646373)
Net worth: 12836.613134967622 (Max net worth: 17114.474096471095)
Profit: 2836.6131349676216
Step: 2004
Balance: 9524.484236697228
Shares held: 46 (Total sold: 5938)
Avg cost for held shares: 50.55212892579608 (Total sales value: 157850.20113646373)
Net worth: 12798.184022916452 (Max net worth: 17114.474096471095)
Profit: 2798.184022916452
Step: 2005
Balance: 9524.484236697228
Shares held: 46 (Total sold: 5938)
Avg cost for held shares: 50.55212892579608 (Total sales value: 157850.20113646373)
Net worth: 12819.417763673795 (Max net worth: 17114.474096471095)
Profit: 2819.417763673795
Step: 2006
Balance: 9524.484236697228
Sha

Shares held: 0 (Total sold: 5984)
Avg cost for held shares: 0 (Total sales value: 161207.29291489048)
Net worth: 12881.57601512397 (Max net worth: 17114.474096471095)
Profit: 2881.57601512397
Step: 2047
Balance: 12881.57601512397
Shares held: 0 (Total sold: 5984)
Avg cost for held shares: 0 (Total sales value: 161207.29291489048)
Net worth: 12881.57601512397 (Max net worth: 17114.474096471095)
Profit: 2881.57601512397
Step: 2048
Balance: 12881.57601512397
Shares held: 0 (Total sold: 5984)
Avg cost for held shares: 0 (Total sales value: 161207.29291489048)
Net worth: 12881.57601512397 (Max net worth: 17114.474096471095)
Profit: 2881.57601512397
Step: 2049
Balance: 12881.57601512397
Shares held: 0 (Total sold: 5984)
Avg cost for held shares: 0 (Total sales value: 161207.29291489048)
Net worth: 12881.57601512397 (Max net worth: 17114.474096471095)
Profit: 2881.57601512397
Step: 2050
Balance: 12881.57601512397
Shares held: 0 (Total sold: 5984)
Avg cost for held shares: 0 (Total sales value